In [1]:
from oddsCheckerScraper import scrapeOddsChecker
from oddsCheckerScraper import bookmakers
from SmarketsRaceBuilder import smarketsRaceBuilder
import betfairClient

In [ ]:
smarketsPrices, urls = smarketsRaceBuilder(8)

In [ ]:
for x in urls:
    print(x)

In [ ]:
betCl = betfairClient.betFairClient()
betCl.login()
betFairPrices, urls = betCl.getAllHorsePrices(24)

In [ ]:
for x in urls:
    print(x)

In [ ]:
oddsCheckerPrices = scrapeOddsChecker(urls, bookmakers, False)

In [ ]:
for price in betFairPrices:
    price.printPrice()

In [ ]:
for price in oddsCheckerPrices:
    price.printPrice()

In [ ]:
len(smarketsPrices)

In [ ]:
for i in range(0,len(smarketsPrices)):
    print(i)
    smarketsPrices[i].getInfo()

In [ ]:
betFairPrices[5].printPrice()
oddsCheckerPrices[0].printPrice()

In [2]:
#race comparison, function to take a list of smarkets horse racing markets and a list of bookie horse racing markets
def compareRaces(exchange_market, bookies_market):
    if exchange_market.prices[0] == None or exchange_market.prices[0][1] == None:
        return (None, "did not have to winner price")
    
    exchange_oddsBack, exchange_OddsLay = exchange_market.prices[0]
    winExchangeOdds = 0.5 * exchange_oddsBack + 0.5 * exchange_OddsLay
    
    placeOdds = exchange_market.prices[bookies_market.ewPlaces - 1]
    
    if placeOdds == None:
        #inverselist = exchange_market.prices[::-1]
        #estimatedPrice= next(item for item in a if item is not None)
        #new_place, exchange_placeOddsBack, exchange_placeOddsLay = estimatedPrice
        #new_place += 1
        #note = ("used odds for place " + str(new_place) + " instead of " + str(bookies_market.ewPlaces))
        return (None, "did not have to place price", "NoValue")
    elif bookies_market.price == "SP":
        return (None, "did not have to bookie price" , "NoValue")
    else:
        exchange_placeOddsBack, exchange_placeOddsLay = placeOdds
        note = "used correct place odds"
        placeExchangeOdds = 0.5 * exchange_placeOddsBack + 0.5 * exchange_placeOddsLay #exchange_placeOddsLay
    
        winProbs = 1/winExchangeOdds
        placeProbs = 1/placeExchangeOdds
        placeOnlyProbs = placeProbs - winProbs

        bookieWin = bookies_market.price
        bookiePlace = ((bookieWin - 1)/bookies_market.ewDenominator) + 1

        winProfit = bookieWin + bookiePlace - 2
        placeProfit = bookiePlace - 2
        loseProfit = -2

        expectedProfit = winProbs * winProfit + placeOnlyProbs * placeProfit + (1 - placeProbs) * (-2)
        expectedValue = (expectedProfit/2)
        
        if expectedValue > 0:
            return (expectedValue, note, "VALUE!") 
        else:
            return (expectedValue, note, "NoValue") 
        
        
def matchRaces(bookieRaces, exchangeRaces):
    paired_list = []
    for exchangePrice in exchangeRaces:
        bookiePrices = [x for x in bookieRaces if x.name.lower() == exchangePrice.name] 
        if not bookiePrices:
            continue 
        else:
            paired_list.append([exchangePrice, bookiePrices])
            
    return paired_list

In [3]:
class valuePrice:
    def __init__(self, name = None, bookmaker = None, event_name =  None, event_date = None,
                 currentPrice = None, exchangeWinPrices = None, exchangePlacePrices = None,
                 places = None, value = None):
        self.name = name
        self.bookie = bookmaker
        self.event_name = event_name
        self.event_date = event_date
        self.currentPrice = currentPrice
        self.exchangeWinPrices = exchangeWinPrices
        self.exchangePlacePrices = exchangePlacePrices
        self.places = places
        self.value = value
        
    def printInfo(self):
        print("---> Name: ", self.name,
              "\n|Bookmaker: ", self.bookie,
              "\n|Event name: ",self.event_name,
              "\n|Event date: ",self.event_date,
              "\n|Current bookie price: ", self.currentPrice,
              "\n|Current exchange win price: ", self.exchangeWinPrices,
              "\n|Current exchange place price: ", self.exchangePlacePrices,
              "\n|Value: ", self.value)
        
        
def checkValueRaces(bookieRaces, exchangeRaces):
    assessedMarketList = []
    valueFinds = []
    matchedRaces = matchRaces(bookieRaces, exchangeRaces)
    for matchedExchangePrice in matchedRaces:
        exchP = matchedExchangePrice[0]
        #print("\n\nExchange price:")
        for bookP in matchedExchangePrice[1]:
            value = compareRaces(exchP, bookP)
            assessedMarketList.append((exchP.name, exchP.event_name, bookP.bookie, value))
            if value[0] != None and value[0] > 0:
                currentPrice = bookP.price
                newValuePrice = valuePrice(exchP.name,bookP.bookie,exchP.event_name, exchP.date, bookP.price,
                                          exchP.prices[0], exchP.prices[bookP.ewPlaces - 1],bookP.ewPlaces, value)
                valueFinds.append(newValuePrice)
            
            
    return valueFinds, newValuePrice

In [ ]:
vals, allVals = checkValueRaces(oddsCheckerPrices, betFairPrices)

In [ ]:
for val in vals:
    val.printInfo()

In [4]:
def runValueFinder(h_thresh):
    
    betCl = betfairClient.betFairClient()
    betCl.login()
    betFairPrices, urls = betCl.getAllHorsePrices(h_thresh)
    
    for url in urls:
        print(url)
        
    oddsCheckerPrices = scrapeOddsChecker(urls, bookmakers, False, False)
    if len(urls) == 0:
        print("no races within time frame")
        
        return betFairPrices, oddsCheckerPrices, None
    else:
        print("Here")
        valueFinds, allMarkets = checkValueRaces(oddsCheckerPrices, betFairPrices)
        
        return betFairPrices, oddsCheckerPrices, valueFinds, allMarkets

In [5]:
betFairPrices, oddsCheckerPrices, valueFinds, allMarkets = runValueFinder(12)

Getting horses for event Southwell
Getting horses for event Kelso
Getting horses for event Chelmsford City
https://www.oddschecker.com/horse-racing/Chelmsford-City/19:00/winner
https://www.oddschecker.com/horse-racing/Chelmsford-City/18:00/winner
https://www.oddschecker.com/horse-racing/Chelmsford-City/19:30/winner
https://www.oddschecker.com/horse-racing/Chelmsford-City/20:30/winner
https://www.oddschecker.com/horse-racing/Chelmsford-City/18:30/winner
https://www.oddschecker.com/horse-racing/Chelmsford-City/20:00/winner
Scraping  1  of  6  urls.


C:\Users\User\Desktop\Learning\Betting\trading-horse-racing\oddsCheckerScraper.py:39: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 39 of the file C:\Users\User\Desktop\Learning\Betting\trading-horse-racing\oddsCheckerScraper.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(prettyHTML)


Scraping  2  of  6  urls.
Scraping  3  of  6  urls.
Scraping  4  of  6  urls.
Scraping  5  of  6  urls.
Scraping  6  of  6  urls.
Here


In [ ]:
for value in valueFinds:
    value.printInfo()